# **Importing Libraries And Reading All Files From Drive**

In [28]:
#importing libraries
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os
import nltk

from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import cmudict
import string
import re
from nltk.corpus import stopwords


In [ ]:
#Reading the Excel file using pandas:
from google.colab import drive
drive.mount('/content/drive')
#please keep all the files in a folder in google drive named"test" before running the following program
file_path = '/content/drive/MyDrive/test/Input.xlsx'
#create a dataframe  of data in excel file
df1 = pd.read_excel(file_path)
print(df1)


In [90]:
#Check All the folders and files in test folder
print(os.listdir('/content/drive/MyDrive/test/'))

['Input.xlsx', 'Text Analysis.docx', 'Objective.docx', 'StopWords', 'MasterDictionary', 'Output', 'CleanedOutput', 'Output Data Structure.xlsx', 'FinalOutput.xlsx']


In [14]:
#allocating all the paths


# Ensure output directory exists in google drive
output_folder = '/content/drive/MyDrive/test/Output/'
# If output directory does't exists in google drive it will create a new directory
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
     # Create a folder in the root directory
    !mkdir -p "/content/drive/My Drive/test/Output/"



stopwords_folder_path = '/content/drive/MyDrive/test/StopWords/'
cleaned_files_folder_path = '/content/drive/MyDrive/test/CleanedOutput/'

# Ensure the cleaned_files folder exists
if not os.path.exists(cleaned_files_folder_path):
    os.makedirs(cleaned_files_folder_path)
        # Create a folder in the root directory
    !mkdir -p "/content/drive/My Drive/test/CleanedOutput/"

Master_Dictionary = '/content/drive/MyDrive/test/MasterDictionary/'
stopwords_folder_path = os.path.join(Master_Dictionary, '/content/drive/MyDrive/test/StopWords/')
positive_file_path = os.path.join(Master_Dictionary, 'positive-words.txt')
negative_file_path = os.path.join(Master_Dictionary, 'negative-words.txt')
Output_Data_Structure = '/content/drive/MyDrive/test/Output Data Structure.xlsx'


# **Extracting Articles From Given URL And Save Them In Output Folder In Drive**

In [ ]:



# Loop through each row in the DataFrame
for index, row in df.iterrows():
    url_id_str = str(row['URL_ID']).lstrip("\\") #"URL_ID" is the coloumn name in excel file
    if url_id_str.endswith('.0'):
        url_id_str = str(int(float(url_id_str)))

    url_link = row['URL']  #"URL" is the coloumn name in excel file

    # Fetch the webpage content
    response = requests.get(url_link)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Extract title and content
    #Parse the content using BeautifulSoup to extract the title and article text
    # Assuming the webpage has title inside <title> tag and article text inside <p> tags
    title = soup.title.string if soup.title else "No Title"
    article_texts = soup.find_all('p')
    article_content = '\n'.join([text.get_text() for text in article_texts])

    # Save to a text file in the outpur folder
    file_path = os.path.join(output_folder, f"{url_id_str}.txt")
    print(f"Saving to {file_path}")  # Print the path where we're saving to confirm
    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(title + '\n' + article_content)

# **Cleaning using Stop Words Lists And Saving Cleaned Files In A Folder "CleanedOutput"**

In [ ]:
#Cleaning using Stop Words Lists


# List all stopwords and output files
stopwords_files = [f for f in os.listdir(stopwords_folder_path) if os.path.isfile(os.path.join(stopwords_folder_path, f))]
output_files = [f for f in os.listdir(output_folder) if os.path.isfile(os.path.join(output_folder, f))]

# Dictionary to store counters
counters = {}

# For each file in the output folder
for output_file in output_files:
    content_file_path = os.path.join(output_folder, output_file)

    with open(content_file_path, 'r', encoding='ISO-8859-1', errors='ignore') as f:
        content = f.read().split()  # Splitting into words for easier counting

    # Iteratively clean the content using each stopwords file
    for stopwords_file in stopwords_files:
        with open(os.path.join(stopwords_folder_path, stopwords_file), 'r', encoding='ISO-8859-1', errors='ignore') as f:
            stopwords = set(f.read().splitlines())

        # Count occurrences of each stopword in the content
        found_stopwords = sum(1 for word in content if word.lower() in stopwords)
        counters[(output_file, stopwords_file)] = found_stopwords

        content = [word for word in content if word.lower() not in stopwords]

    # Save the final cleaned content to the cleaned_files folder
    cleaned_content_file_path = os.path.join(cleaned_files_folder_path, output_file)
    with open(cleaned_content_file_path, 'w', encoding='ISO-8859-1') as f:
        f.write(' '.join(content))

# Print the summary
for (output_file, stopwords_file), count in counters.items():
    print(f"In {output_file}, found {count} stopwords from {stopwords_file} and they have been cleaned.")

print("All files processed.")


# **Creating Dictionary Of Positive And Negative Words**

In [ ]:
# Define paths to your files



# Function to read words from a file into a list
def load_words_from_file(file_path):
    with open(file_path, 'r', encoding='utf-8', errors='ignore') as file:
        words = file.read().splitlines()
    return words

# Load all stopwords from every file in the stopwords folder into a single set
stopwords_files = [f for f in os.listdir(stopwords_folder_path) if os.path.isfile(os.path.join(stopwords_folder_path, f))]
stopwords = set()
for stopwords_file in stopwords_files:
    stopwords.update(load_words_from_file(os.path.join(stopwords_folder_path, stopwords_file)))

# Load positive and negative words, filtering out stopwords
positive_words = [word for word in load_words_from_file(positive_file_path) if word.lower() not in stopwords]
negative_words = [word for word in load_words_from_file(negative_file_path) if word.lower() not in stopwords]

# Combine into a dictionary
words_dict = {
    'positive': positive_words,
    'negative': negative_words
}

# Optional: Print the dictionary to see the result
print(words_dict)


# **Extracting Derived variables**

In [ ]:

 nltk.download('punkt')

# Data dictionary to store the scores
data = {
    'URL_ID': [],
    'POSITIVE SCORE': [],
    'NEGATIVE SCORE': [],
    'POLARITY SCORE': [],
    'SUBJECTIVITY SCORE': []
}

# Tokenize and calculate scores
for file_name in os.listdir(cleaned_files_folder_path):
    file_path = os.path.join(cleaned_files_folder_path, file_name)

    with open(file_path, 'r', encoding='utf-8', errors='ignore') as file:
        text = file.read()

    tokens = word_tokenize(text.lower())  # Convert to lowercase before tokenizing

    POSITIVE_SCORE = sum(1 for token in tokens if token in words_dict['positive'])
    NEGATIVE_SCORE = sum(1 for token in tokens if token in words_dict['negative'])

    POLARITY_SCORE = round((POSITIVE_SCORE - NEGATIVE_SCORE) / ((POSITIVE_SCORE + NEGATIVE_SCORE )+ 0.000001),2) #using round function it rounds the number to two decimal places.
    SUBJECTIVITY_SCORE =round( (POSITIVE_SCORE + NEGATIVE_SCORE) / (len(tokens) + 0.000001),2)  #using round function it rounds the number to two decimal places.

    # Extract URL ID from the file name (assuming it's the entire name without the extension)
    URL_ID = os.path.splitext(file_name)[0]

    # Save the data
    data['URL_ID'].append(URL_ID)
    data['POSITIVE SCORE'].append(POSITIVE_SCORE)
    data['NEGATIVE SCORE'].append(NEGATIVE_SCORE)
    data['POLARITY SCORE'].append(POLARITY_SCORE)
    data['SUBJECTIVITY SCORE'].append(SUBJECTIVITY_SCORE)

# Convert the data dictionary into a pandas DataFrame
df2 = pd.DataFrame(data)
print(df2)


# **Analysis of Readability**

In [ ]:


# Downloading required NLTK data
nltk.download('punkt')
nltk.download('cmudict')

d = cmudict.dict()

# Helper function to count syllables in a word
def nsyl(word):
    if word.lower() in d:
        return max([len(list(y for y in x if y[-1].isdigit())) for x in d[word.lower()]])
    return 0

# Function to calculate the Gunning Fog index
def gunning_fog(text):
    sentences = sent_tokenize(text)
    words = word_tokenize(text)

    num_sentences = len(sentences)
    num_words = len(words)

    if num_words == 0:
        return None, None, None  # avoid division by zero

    average_sentence_length = round((num_words / num_sentences),2)
    complex_words = [word for word in words if nsyl(word) >= 3]
    percentage_complex = round(((len(complex_words) / num_words) * 100),2)
    fog_index = round((0.4 * (average_sentence_length + percentage_complex)),2)

    return average_sentence_length, percentage_complex, fog_index



# Data dictionary to store the analysis results
data = {
    'URL_ID': [],
    'AVG SENTENCE LENGTH': [],
    'PERCENTAGE OF COMPLEX WORDS': [],
    'FOG INDEX': []
}

# Process each file in the CleanedOutput folder
for file_name in os.listdir(cleaned_files_folder_path):
    file_path = os.path.join(cleaned_files_folder_path, file_name)
    with open(file_path, 'r', encoding='utf-8', errors='ignore') as file:
        text = file.read()

    avg_sent_len, perc_complex, fog_idx = gunning_fog(text)
    if avg_sent_len is not None:  # check to avoid appending when num_words = 0
        data['URL_ID'].append(os.path.splitext(file_name)[0])
        data['AVG SENTENCE LENGTH'].append(avg_sent_len)
        data['PERCENTAGE OF COMPLEX WORDS'].append(perc_complex)
        data['FOG INDEX'].append(fog_idx)

# Convert data dictionary to a DataFrame
df3 = pd.DataFrame(data)
print(df3)


# **Average Number of Words Per Sentence**

In [ ]:



# Function to calculate the Average Number of Words Per Sentence
def avg_words_per_sentence(text):
    sentences = sent_tokenize(text)
    words = word_tokenize(text)

    num_sentences = len(sentences)
    num_words = len(words)

    if num_sentences == 0:
        return None  # to avoid division by zero

    return round((num_words / num_sentences),2)


# Data dictionary to store the results
data = {
    'URL_ID': [],
    'AVG NUMBER OF WORDS PER SENTENCE': []
}

# Process each file in the CleanedOutput folder
for file_name in os.listdir(cleaned_files_folder_path):
    file_path = os.path.join(cleaned_files_folder_path, file_name)
    with open(file_path, 'r', encoding='utf-8', errors='ignore') as file:
        text = file.read()

    avg_wps = avg_words_per_sentence(text)
    if avg_wps is not None:  # to avoid appending when num_sentences = 0
        data['URL_ID'].append(os.path.splitext(file_name)[0])
        data['AVG NUMBER OF WORDS PER SENTENCE'].append(avg_wps)

# Convert data dictionary to a DataFrame
df4 = pd.DataFrame(data)
print(df4)


# **Complex Word Count**

In [50]:


# Create a dictionary for the syllable count
d = cmudict.dict()

# Helper function to count syllables in a word
def nsyl(word):
    if word.lower() in d:
        return max([len(list(y for y in x if y[-1].isdigit())) for x in d[word.lower()]])
    return 0

# Function to count complex words in a text
def count_complex_words(text):
    words = word_tokenize(text)
    complex_words = [word for word in words if nsyl(word) > 2]
    return len(complex_words)



# Data dictionary to store the results
data = {
    'URL_ID': [],
    'COMPLEX WORD COUNT': []
}

# Process each file in the CleanedOutput folder
for file_name in os.listdir(cleaned_files_folder_path):
    file_path = os.path.join(cleaned_files_folder_path, file_name)
    with open(file_path, 'r', encoding='utf-8', errors='ignore') as file:
        text = file.read()

    complex_word_count = count_complex_words(text)
    data['URL_ID'].append(os.path.splitext(file_name)[0])
    data['COMPLEX WORD COUNT'].append(complex_word_count)

# Convert data dictionary to a DataFrame
df5 = pd.DataFrame(data)
print(df5)


      URL_ID  COMPLEX WORD COUNT
0        123                 390
1        321                 165
2       2345                 243
3       4321                 295
4        432                 295
..       ...                 ...
109    50921                 122
110  51382.8                 279
111  51844.6                 343
112  52306.4                 263
113  52768.2                 208

[114 rows x 2 columns]


# **Word Count**

In [ ]:
nltk.download('stopwords')

# Set up stopwords and punctuation filter
stop_words = set(stopwords.words('english'))
punctuations = set('?!.,')

# Function to count cleaned words in a text
def count_cleaned_words(text):
    words = word_tokenize(text)
    cleaned_words = [word for word in words if word.lower() not in stop_words and all(char not in punctuations for char in word)]
    return len(cleaned_words)



# Data dictionary to store the results
data = {
    'URL_ID': [],
    'WORD COUNT': []
}

# Process each file in the CleanedOutput folder
for file_name in os.listdir(cleaned_files_folder_path):
    file_path = os.path.join(cleaned_files_folder_path, file_name)
    with open(file_path, 'r', encoding='utf-8', errors='ignore') as file:
        text = file.read()

    cleaned_word_count = count_cleaned_words(text)
    data['URL_ID'].append(os.path.splitext(file_name)[0])
    data['WORD COUNT'].append(cleaned_word_count)

# Convert data dictionary to a DataFrame
df6 = pd.DataFrame(data)
print(df6)


# **Syllable Count Per Word**

In [ ]:




def count_syllables(word):
    vowels = "aeiouAEIOU"

    # Convert word to lowercase
    word = word.lower()

    # Count the vowels
    count = sum(1 for char in word if char in vowels)

    # Adjust for exceptions
    if word.endswith("es") or word.endswith("ed"):
        count -= 1

    # Ensure we have at least one syllable
    return max(1, count)


# Data dictionary to store the results
data = {
    'URL_ID': [],
    'Word': [],
    'SYLLABLE PER WORD': []
}

# Process each file in the folder
for file_name in os.listdir(cleaned_files_folder_path):
    file_path = os.path.join(cleaned_files_folder_path, file_name)
    with open(file_path, 'r', encoding='utf-8', errors='ignore') as file:
        text = file.read()

    words = word_tokenize(text)
    for word in words:
        syllable_count = count_syllables(word)
        data['URL_ID'].append(os.path.splitext(file_name)[0])
        data['Word'].append(word)
        data['SYLLABLE PER WORD'].append(syllable_count)

# Convert data dictionary to a DataFrame
df7 = pd.DataFrame(data)
print(df7)


# **Personal Pronouns**

In [ ]:


def count_personal_pronouns(text):
    # Tokenize the text
    words = word_tokenize(text)

    # Define regex pattern for personal pronouns, ignoring case (i.e., both 'I' and 'i' will match)
    pattern = r'\b(?:i|we|my|ours|us)\b'

    # Join words into a single string to match against pattern and handle the 'US' country exception
    joined_text = ' '.join([word for word in words if word.lower() != 'us' or word.islower()])

    return len(re.findall(pattern, joined_text, flags=re.IGNORECASE))



# Data dictionary to store the results
data = {
    'URL_ID': [],
    'PERSONAL PRONOUNS': []
}

# Process each file in the folder
for file_name in os.listdir(cleaned_files_folder_path):
    file_path = os.path.join(cleaned_files_folder_path, file_name)
    with open(file_path, 'r', encoding='utf-8', errors='ignore') as file:
        text = file.read()

    pronoun_count = count_personal_pronouns(text)
    data['URL_ID'].append(os.path.splitext(file_name)[0])
    data['PERSONAL PRONOUNS'].append(pronoun_count)

# Convert data dictionary to a DataFrame
df8 = pd.DataFrame(data)
print(df8)


# **Average Word Length**

In [ ]:


def average_word_length(text):
    # Tokenize the text
    words = word_tokenize(text)

    # Compute total characters and total words
    total_characters = sum(len(word) for word in words)
    total_words = len(words)

    # To avoid division by zero
    if total_words == 0:
        return 0

    return total_characters / total_words



# Data dictionary to store the results
data = {
    'URL_ID': [],
    'AVG WORD LENGTH': []
}

# Process each file in the folder
for file_name in os.listdir(cleaned_files_folder_path):
    file_path = os.path.join(cleaned_files_folder_path, file_name)
    with open(file_path, 'r', encoding='utf-8', errors='ignore') as file:
        text = file.read()

    avg_word_len = round(average_word_length(text),2)
    data['URL_ID'].append(os.path.splitext(file_name)[0])
    data['AVG WORD LENGTH'].append(avg_word_len)

# Convert data dictionary to a DataFrame
df9 = pd.DataFrame(data)
print(df9)


# **Merging All DataFrames In One**

In [ ]:
dataframes =  [df1, df2, df3, df4, df5, df6, df8, df9] # and so on, or dynamically create this list

# Convert the 'URL ID' column of each dataframe to string
for df in dataframes:
    df['URL_ID'] = df['URL_ID'].astype(str)

final_df = dataframes[0]
for df in dataframes[1:]:
    final_df = final_df.merge(df, on="URL_ID", how="inner")

print(final_df)

# **Converting Final Dataframe To Excel And Saving Output In Drive**

In [89]:


# Path in Google Drive
file_path = '/content/drive/MyDrive/test/FinalOutput.xlsx'

# Save the dataframe to Excel
final_df.to_excel(file_path, engine='openpyxl', index=False)
